## Предобработка и анализ данных

Загружаем модули

In [13]:
#!pip install nltk
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


Загружаем сами данные

In [14]:
meta = pd.read_json('datasets/meta.json', lines=True)
covers = pd.read_json('datasets/covers.json', lines=True)
lyrics = pd.read_json('datasets/lyrics.json', lines=True)

Объединим данные в единый датасет

In [15]:
df = meta.merge(lyrics, left_on = 'track_id', right_on = 'track_id', how='outer')
df = df.merge(covers, left_on = 'track_id', right_on = 'track_id', how='outer')
df.drop('lyricId', axis= 1 , inplace= True )

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72906 entries, 0 to 72905
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           72905 non-null  object 
 1   dttm               72905 non-null  float64
 2   title              72905 non-null  object 
 3   language           22870 non-null  object 
 4   isrc               72566 non-null  object 
 5   genres             72905 non-null  object 
 6   duration           72905 non-null  float64
 7   text               11414 non-null  object 
 8   original_track_id  5378 non-null   object 
 9   track_remake_type  72571 non-null  object 
dtypes: float64(2), object(8)
memory usage: 5.6+ MB


Мы видим большое количество пропусков в некоторых колонках. К сожалению, это те данные, с которыми мы будем работать. Для построения и оценки алгоритма группирования нам необходимы размеченные данные с текстом, потому заполнять пропуски в этих колонках мы не можем, но можем заполнить пропуски в колонке language.

In [16]:
df2 = df.copy()
df2 = df2[['language', 'original_track_id']].dropna() #Создаём временный датасет, где есть только размеченные данные, но не заполняем пропуски 
#в языке
df['language'] = df['language'].fillna('Unknown')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72906 entries, 0 to 72905
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           72905 non-null  object 
 1   dttm               72905 non-null  float64
 2   title              72905 non-null  object 
 3   language           72906 non-null  object 
 4   isrc               72566 non-null  object 
 5   genres             72905 non-null  object 
 6   duration           72905 non-null  float64
 7   text               11414 non-null  object 
 8   original_track_id  5378 non-null   object 
 9   track_remake_type  72571 non-null  object 
dtypes: float64(2), object(8)
memory usage: 5.6+ MB


### Подготовим данные обучения модели

Избавимся от пропусков, оставим только размеченные данные. Проверим данные на дубликаты по столбцу track_id и избавимся от них, если надо. Колонка genres нам пока не понадобится, так что от неё тоже избавимся. 

In [17]:
data = df.copy()
data = data.dropna()
data.drop('genres', axis= 1 , inplace= True )
print("Количество дубликатов: ",data.duplicated(subset='track_id').sum())
print()
data = data.drop_duplicates(subset='track_id')
#data = data.query('language == "EN"')
data.reset_index(drop=True, inplace=True)

data.info()

Количество дубликатов:  541

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           3000 non-null   object 
 1   dttm               3000 non-null   float64
 2   title              3000 non-null   object 
 3   language           3000 non-null   object 
 4   isrc               3000 non-null   object 
 5   duration           3000 non-null   float64
 6   text               3000 non-null   object 
 7   original_track_id  3000 non-null   object 
 8   track_remake_type  3000 non-null   object 
dtypes: float64(2), object(7)
memory usage: 211.1+ KB


## Построение модели

По сути, наша модель будет представлять собой алгоритм, который сможет искать в заданном датасете наиболее похожие тексты. Для этого будем использовать модуль nltk. 

Мы расчитаем расстояние нашего трека со всеми остальными в датасете. Группу треков будем составлять исходя из близости текстов треков. Те треки, которые ближе определённого значения, находятся в нашей группе и среди них есть каверы и оригинал (который мы будем считать по дате).

По сути, обучение модели сводится к подбору оптимального порога.



In [18]:

#Функция, рассчитывающая расстояние между двумя текстами
def cosine_sim(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

#Функция, собственно выполняющая расчёты
def distantions(data_frame = data, my_text = '', tresh_hold = -1, column_name_ = 'text'):
    #Функция принимает два параметра - датафрейм, с которым мы будем иметь дело и текст трека, с которым мы будем сравнивать всё остальное.
    #По умолчанию, значение data_frame = data, но можно использовать любой датафрейм. Параметр column_name_ указывает название колонку
    #с текстами песен. Можно также анализировать по названию, тогда надо указать соответствующую колонку
    df_ = data_frame.copy()
   
    df_['sim'] = 0 #Создаём столбец, куда будем записывать расстояния нашего текста с остальными.
    df_['sim'] = df_['sim'].astype(float)
    text_1 = my_text
    
    #Перебираем все возможные тексты в датафрейме
    for i in range(len(df_)):
        try:
            text_2 = df_.loc[i,column_name_]  
            corpus = [text_1, text_2]
            #Присваиваем i-той строке столбца dist значение, равное расстоянию нашего текста до его текста
            df_.loc[i, 'sim'] = cosine_sim(text_1, text_2)
            
        except:
            ...
    #Возвращаем исходный датафрейм с новым столбцом
    if tresh_hold == -1:
        return df_
    else:
        df_ = df_.query('sim >= @tresh_hold')
        return df_ 
        
    

### Обучение модели


Теперь создадим функцию, которая будет выдавать нам метрику для каждого конкретного трека. В качестве параметров она будет принимать три значения:
- df1 - датасет, в котором уже размечены расстояния от исходного трека до всех остальных
- tid - Номер строки трека, с которым мы сравниваем в датасете, который мы передаём. 
- tresh_hold - пороговое сравнение. 

Это работает так: когда мы используем функцию, возвращающую схожесть разных текстов, мы считаем, что к нашей группе относятся тексты, которые схожи больше, чем на определённое значение. Эта функция высчитывает абсолютное значение ложноположительных и ложноотрицательных результатов.

Это значит, она считает сколько треков, не относящейся к нашей группе, модель относит к ней при заданном пороге и наоборот, сколько треков относящихся к нашей группе, модель относит к группе.

Напомню, что под группой мы подразумеваем треки, которые являются оригиналом и его каверами. В размеченных данных это те треки, у которых одинаковый original_track_id.

In [19]:
def find_metric(df1, tid, tresh_hold):
    DF_METRIC = df1.copy()
    TRAC_ID = DF_METRIC.loc[tid,'track_id']
    otid =  DF_METRIC.loc[tid, 'original_track_id']
    METRIC = len(DF_METRIC.query('original_track_id == @otid'))
    false_positive = len(DF_METRIC.query('sim >= @tresh_hold & original_track_id != @otid'))  #Количество ложноположительных результатов
    false_negative = len(DF_METRIC.query('sim < @tresh_hold & original_track_id == @otid')) # Количество ложноотрицательных результатов
    false_positive_rel = false_positive / METRIC
    false_negative_rel = false_negative / METRIC
    true_positive = len(DF_METRIC.query('sim >= @tresh_hold & original_track_id == @otid'))
    true_negative = len(DF_METRIC.query('sim < @tresh_hold & original_track_id != @otid')) 
    classic_precision = true_positive / (true_positive + false_positive)
    classic_recall = true_positive / (true_positive + false_negative)
    F1 = 2 * (classic_precision * classic_recall) / (classic_precision + classic_recall)
    return false_positive, false_negative, false_positive_rel, false_negative_rel, classic_precision, classic_recall, F1, TRAC_ID
    

Теперь нам нужно собственно расчитать оптимальный порог. Это и есть единственный параметр нашей модели. Для этого возьмём список разных порогов,
рассчитаем метрики для каждого значения и сравним. Считать мы будем по всему тренировочному датасету, в качестве итогового результата будут СРЕДНИЕ значения ошибок. То есть, мы узнаем, сколько в среднем наша модель выдаёт ложных срабатываний.


In [20]:
tresh_hold = [0, 0.5,0.6, 0.7, 0.75, 0.8, 0.9] #значения порогов
LIST = [] #Создаём пустой список
#Превращаем его в список пустых списков. Да, я пробовал LIST = [[]] * len(tresh_hold), но в таком случае дальнейшая процедура работает 
#некорректно, потому приходится именно так извращаться и делать это в цикле.
for i in range(len(tresh_hold)):

   LIST.append([])




In [22]:
#В данном цикле считаем метрики для КАЖДОГО трека с КАЖДЫМ значением порога. Результат записываем в наш список списков. 
#Соответственно, в каждом подсписке будут результаты для всего датасета с соответствующим пороговым значением. 
for i in tqdm(range(data.shape[0])):
    result = distantions(data_frame = data, my_text = data.loc[i, 'text'], tresh_hold = -1)
    for i2 in range(len(tresh_hold)):
        cortege = find_metric(result, i, tresh_hold[i2])
        
        LIST[i2].append(cortege)
        
            
            



  0%|          | 0/3000 [00:00<?, ?it/s]

In [23]:
#Теперь преобразуем списки в датафреймы и считаем средние ошибки.
Metrics = []

for i in range(len(tresh_hold)):
   
    DFERR = pd.DataFrame(LIST[i], columns = ['FP', 'FN', 'FPrel', 'FNrel','P', 'R', 'F1', 'ID'])

    Metrics.append([tresh_hold[i], DFERR['FP'].mean(), DFERR['FN'].mean(), DFERR['FPrel'].mean(), DFERR['FNrel'].mean(), DFERR['P'].mean(), DFERR['R'].mean(), DFERR['F1'].mean()])
    #print('Treshhold:', tresh_hold[i], 'FP:',  DFERR['FP'].mean(), 'FN:',   'FP_rel:', DFERR['FPrel'].mean(), 'FN_rel', DFERR['FNrel'].mean())

Metrics = pd.DataFrame(Metrics, columns = ['Treshhold', 'FP_abs', 'FN_abs', 'FP_rel', 'FN_rel', 'Precision', 'Recall', 'F1'])
Metrics


,Treshhold,FP_abs,FN_abs,FP_rel,FN_rel,Precision,Recall,F1
0,0.00,2997.874000,0.000000,2781.000000,0.000000,0.000709,1.000000,0.001412
1,0.50,1.577333,0.146667,1.331448,0.009588,0.854366,0.990412,0.877218
2,0.60,0.370667,0.264667,0.316758,0.015051,0.934367,0.984949,0.940366
3,0.70,0.194667,0.382667,0.160568,0.021213,0.957449,0.978787,0.954147
4,0.75,0.148000,0.464000,0.119756,0.026378,0.963735,0.973622,0.955039
5,0.80,0.118667,0.548667,0.094023,0.030713,0.968270,0.969287,0.955073
6,0.90,0.074000,0.790667,0.059029,0.045477,0.975493,0.954523,0.947067


Мы посчитали метрики для разных пороговых значений. В качестве метрики возьмём F1 метрику. Она считается таким образом: мы считаем метрику для каждого конкретного трека, то есть то, каким образом алгоритм определил кластер для конкретного трека, а затем считается усреднённое значение метрик. 

Напишем функцию, которая выведет список всех "похожих" треков из заданного датасета, а также определит оригинал.

In [24]:
BEST_TRESHHOLD = 0.8
CLEAR = ['track_id', 'title', 'text', 'year']

#df_tracks -  
def grouping_tracks(
track_id, #Принимает на вход идентификатор трека (ОБЯЗАТЕЛЬНЫЙ ПАРАМЕТР)
df_tracks = df, #Датафрейм, в котором будет проводиться поиск (ОБЯЗАТЕЛЬНЫЙ ПАРАМЕТР) 
method = 'text', #Метод, по которому будет проводиться поиск: 
                 #text - только по текстам песен
                 #title - только по названиям песен
                 #both - функция будет искать соотвествия как в названиях, так и в текстах. Треки с текстом будут выше в выдаче, они в любом
                 #случае будут считаться более релевантными
original = 'year', #Метод определения оригинала:
                     #year 
tresh_hold_ = BEST_TRESHHOLD, #Параметр, который указывает какой порог использовать при поиске
clear = True #Если True, выводит только несколько столбцов, иначе все
):
    df_tracks_ = df_tracks.copy()
    if len(df_tracks.query('track_id == @track_id')) == 0:
        return 'This music track was not found'
    df_tracks_['year'] = df_tracks_['isrc'].str[5:7] #Получаем год регистрации трека

    if method == 'text' or method == 'both':
        txt = df_tracks.query('track_id == @track_id').reset_index(drop = True).loc[0, 'text']
        result = distantions(data_frame = df_tracks_, my_text = txt, tresh_hold = tresh_hold_)
    
    if method == 'title' or method == 'both':
        txt = df_tracks.query('track_id == @track_id').reset_index(drop = True).loc[0, 'title']
        result2 = distantions(data_frame = df_tracks_, my_text = txt, tresh_hold = tresh_hold_, column_name_ = 'title')
        if method == 'both':
            result2['sim'] = result2['sim'].apply(lambda x: x - 2)
            result = pd.concat([result, result2])
            result = result.sort_values(by = 'sim', ascending = True)
            result = result.drop_duplicates(subset='track_id')
            result = result.sort_values(by = 'sim', ascending = False)
        else:
            result = result2
    if original == 'year':
        result['year'] = result['year'].fillna('5-')
        result['year'] = result['year'].replace('5-', '9999')
        result['year'] = result['year'].astype(int)
        result['year'] = result['year'].apply(lambda x: x + 2000 if x <= 24 else x + 1900)
        result_original = result.sort_values(by = 'year', ascending = True).head(1)
    if clear:
        return result_original[CLEAR], result[CLEAR]
    else:
        return result_original, result


In [25]:
a, b = grouping_tracks('deb9b9598176a0bab1212d430b10bd04', df, 'text')

In [26]:
a

,track_id,title,text,year
45447,deb9b9598176a0bab1212d430b10bd04,Sweet Dreams (Are Made of This),Sweet dreams are made of this\nWho am I to dis...,1983


In [27]:
b

,track_id,title,text,year
1564,8295168b2df271a91f9e4f5d6a7aad69,Sweet Dreams (Are Made of This),Sweet dreams are made of this\nWho am I to dis...,2021
17054,6a0060c234c43fcffd4b3ee28621af5a,Sweet Dreams,Sweet dreams are made of this\nWho am I to dis...,2019
20426,08a52b88aa5ffca6b41b58c6d5ec7a52,Sweet Dreams,Sweet dreams are made of this\nWho am I to dis...,2019
20427,08a52b88aa5ffca6b41b58c6d5ec7a52,Sweet Dreams,Sweet dreams are made of this\nWho am I to dis...,2019
22278,8f36faa55a52681c41a6cfa1ff1176f0,Sweet Dreams,Sweet dreams are made of this\nSweet dreams ar...,2017
23161,97f3c02d03bcb3779c148bd060cc3483,Sweet Dreams (Are Made of This),"Sweet, sweet dreams are made of this\nWho am I...",1994
25084,f8fb3c76c159efb0033c3b48e0c2a045,Sweet Dreams,Sweet dreams are made of this\nWho am I to dis...,2021
38375,07cf69ce6c50fe7846e2e90eb05b3aeb,Sweet Dreams,Sweet dreams are made of this\nWho am I to dis...,2021
45447,deb9b9598176a0bab1212d430b10bd04,Sweet Dreams (Are Made of This),Sweet dreams are made of this\nWho am I to dis...,1983
45454,bf8b2ce531f3844f31a147ccca54151b,Sweet Dreams (Are Made of This),Sweet dreams are made of this\nWho am I to dis...,2003
